In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import glob
from scipy import stats
%matplotlib inline

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>")) 

In [3]:
#check sklearn version
# import sklearn
# sklearn.__version__

In [4]:
#Def function to read all XPT data in a file into a dictionary contains all dataframes
def readXPT(path):
    dataset = {}
    path = os.path.join(path)
    filenames = glob.glob(path + '/*.XPT')
    
    for filename in filenames:
        dfname = filename.split('/')[-1]
        df = pd.read_sas(filename)
        df['SEQN'] = df['SEQN'].astype(int)
        df.set_index('SEQN', inplace=True)
        dataset[dfname] = df

    return dataset

In [5]:
#Def function to read all XPT data in a file into a dictionary contains all dataframes
#And report any error(file does not contain 'SEQN' column)
def readXPT2(path):
    dataset = {}
    path = os.path.join(path)
    filenames = glob.glob(path + '/*.XPT')
    
    for filename in filenames:
        try:
            dfname = filename.split('/')[-1]
            df = pd.read_sas(filename)
            df['SEQN'] = df['SEQN'].astype(int)
            df.set_index('SEQN', inplace=True)
            dataset[dfname] = df
        except:
            print(filename)

    return dataset

In [6]:
#Def function to merge all dataframes of XPT files into one dataframe
def mergeXPT(dataset):
    data = pd.concat(dataset.values(), join = 'outer', axis=1, sort=False)
    return data

In [7]:
Questionnaire = readXPT2('/Users/Bing Hu/Desktop/CapstoneData/raw_data/2015-2016/Questionnaire')

/Users/Bing Hu/Desktop/CapstoneData/raw_data/2015-2016/Questionnaire\RXQ_DRUG.xpt


In [8]:
Demo = readXPT('/Users/Bing Hu/Desktop/CapstoneData/raw_data/2015-2016/Demographics')

In [9]:
Exam = readXPT('/Users/Bing Hu/Desktop/CapstoneData/raw_data/2015-2016/Examination')

In [10]:
Lab = readXPT('/Users/Bing Hu/Desktop/CapstoneData/raw_data/2015-2016/Laboratory')

In [11]:
Questionnaire = mergeXPT(Questionnaire)
Demo = mergeXPT(Demo)
Exam = mergeXPT(Exam)
Lab = mergeXPT(Lab)

In [12]:
#Merge data from different folders
All = pd.concat([Questionnaire, Exam, Demo,Lab], axis=1, join='outer', sort= False)

In [13]:
All.shape

(9971, 1451)

In [14]:
#Drop non-labelled samples & Separate Label and Predictor
All = All[(All['MCQ160M']==1) | (All['MCQ160M']==2)]

#Obtain Response Variable
result = All['MCQ160M']
result[result==2] = 0

#Drop features directly related to thyroid
features = All.drop(['MCQ170M','MCQ180M','MCQ240BB','MCQ160M'],axis=1)

features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5708 entries, 83732 to 93702
Columns: 1447 entries, ACD011A to LBDVXYLC
dtypes: float64(1369), object(78)
memory usage: 63.1+ MB


In [15]:
All.to_csv('~/Desktop/Capstone/all_features_1516.csv')